In [88]:
import os
from pathlib import Path
import pandas as pd
#import alpaca_trade_api as tradeapi
import yfinance as yf

In [89]:
csvpath = Path('Resources/Tickers.csv')

ticker = pd.read_csv(csvpath, header=None)

In [90]:
etfs = ticker[0].values


In [54]:
etf_df = pd.DataFrame()

for etf in etfs:
        
    hist = getData(etf)
    
    etf_df[etf] = hist["Close"]    

- ISBT: No data found, symbol may be delisted
- VQN: 1d data not available for startTime=-2208988800 and endTime=1622769538. Only 100 years worth of day granularity data are allowed to be fetched per request.


In [43]:
def getData(ticker):

    tickerData = yf.Ticker(ticker)
    
    tickerData_history = tickerData.history(period="max")
    
    tickerData_history = tickerData_history[tickerData_history.index > '2015-01-01']
        
    return tickerData_history 

In [91]:
etf_df

,TAN,QCLN,XLE,ICLN,AMLP,VDE,XOP,XGD.TO,ZTL.NE,XST.TO,...,FTL,COMT,IYK,PSL,IEV,VDC,FSTA,KXI,RHS,PBJ
Date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,30.974342,16.436447,61.475479,8.547479,51.294609,91.071991,176.731293,9.227556,NaN,42.369911,...,NaN,33.588982,90.918373,47.267120,35.943035,106.012199,24.938875,38.655239,91.709679,28.180506
2015-01-05,30.163071,15.875578,58.932369,8.363379,49.900425,87.459305,165.399994,9.452618,NaN,41.577690,...,NaN,32.806129,89.897110,46.814995,34.864407,105.143532,24.751745,38.039394,91.118721,27.828474
2015-01-06,29.579681,15.533349,58.066620,8.345845,49.232380,86.144150,160.392319,10.020167,NaN,40.480022,...,NaN,32.365780,89.382103,46.833839,34.490707,105.058357,24.726227,37.948322,91.092278,27.652460
2015-01-07,29.588793,15.656926,58.190300,8.424745,49.058098,86.355209,158.783966,9.883172,NaN,41.033630,...,NaN,32.373943,90.892166,47.493191,34.813450,106.787193,25.168530,38.455746,92.327072,28.245346
2015-01-08,30.281570,16.046688,59.496651,8.626377,49.493786,88.222443,164.157227,9.697249,NaN,41.138622,...,NaN,32.585964,92.367332,48.105473,35.348511,108.430862,25.559793,39.093281,93.861755,28.699266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-27,79.320000,62.680000,52.110001,22.809999,34.540001,72.680000,89.040001,20.719999,51.369999,70.040001,...,NaN,33.224998,181.669998,96.540001,54.779999,184.399994,43.080002,61.889999,163.820007,42.540001
2021-05-28,79.110001,62.490002,52.209999,22.920000,34.340000,72.760002,89.070000,20.850000,51.560001,70.519997,...,NaN,33.189999,181.690002,96.220001,54.919998,184.830002,43.150002,61.930000,164.619995,42.750000
2021-06-01,78.769997,63.099998,54.220001,22.870001,35.320000,75.779999,94.830002,20.910000,51.400002,70.040001,...,NaN,33.669998,181.149994,96.989998,55.250000,184.460007,43.070000,61.810001,164.399994,43.160000


In [113]:
testing_pol = getCovidData('covid-19-testing-policy.csv')
contact_tr = getCovidData('covid-contact-tracing.csv')
vax_pol = getCovidData('covid-vaccination-policy.csv')
debt_relief = getCovidData('debt-relief-covid.csv')
face_cover = getCovidData('face-covering-policies-covid.csv')
income_support = getCovidData('income-support-covid.csv')
internal_mov = getCovidData('internal-movement-covid.csv')
int_travel = getCovidData('international-travel-covid.csv')
public_camp = getCovidData('public-campaigns-covid.csv')
public_events = getCovidData('public-events-covid.csv')
public_gather = getCovidData('public-gathering-rules-covid.csv')
public_trans = getCovidData('public-transport-covid.csv')
stay_home = getCovidData('stay-at-home-covid.csv')
work_close = getCovidData('workplace-closures-covid.csv')
school_close = getCovidData('school-closures-covid.csv')
stringency = getCovidData('covid-stringency-index.csv')

covid_data = pd.concat([testing_pol, contact_tr, vax_pol, debt_relief, face_cover, income_support, internal_mov,
                       int_travel, public_camp, public_events, public_gather, public_trans, stay_home, work_close,
                       school_close, stringency
                       ], axis="columns", join="outer")


covid_data

,testing_policy,contact_tracing,vaccination_policy,debt_relief,facial_coverings,income_support,restrictions_internal_movements,international_travel_controls,public_information_campaigns,cancel_public_events,restriction_gatherings,close_public_transport,stay_home_requirements,workplace_closures,school_closures,stringency_index
Day,,,,,,,,,,,,,,,,
2020-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2020-01-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2020-01-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2020-01-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-18,3,1,5,1,4,1,1,3,2,1,4,1,1,1,1.0,52.31
2021-05-19,3,1,5,1,4,1,1,3,2,1,4,1,1,1,1.0,52.31
2021-05-20,3,1,5,1,4,1,1,3,2,1,4,1,1,1,1.0,52.31


In [76]:
def getCovidData(csv):
    
    csvpath = Path('Resources/COVID_Data/' + csv)
    data = pd.read_csv(csvpath, index_col="Day")
    
    cleaned_data = data.loc[data['Code'] == 'USA'].drop(columns=['Entity', 'Code'])
    
    return cleaned_data